# Raise base class events in derived classes

Let's create a smart kitchen that monitors multiple appliances and logs what they're being used for.

In [ ]:
public class ProcessEventArgs : EventArgs {
    public string Message { get; init; }
    public DateTime Start { get; } = DateTime.Now;

    public TimeSpan GetDuration(ProcessEventArgs previousProcess) {
        return Start - previousProcess.Start;
    }
}

In [ ]:
/// Publisher base class
public abstract class Appliance {
    public ProcessEventArgs CurrentProcess { get; set; }
    public List<ProcessEventArgs> ProcessesHistory { get; } = new();
    public Queue<ProcessEventArgs> ProcessesQueue { get; } = new();

    public event EventHandler<ProcessEventArgs> ProcessStartedEvent;
    public event EventHandler<ProcessEventArgs> ProcessEndedEvent;

    /// Derived classes can override and call this method
    protected virtual void OnProcessStarted(ProcessEventArgs e) {
        CurrentProcess = e;
        ProcessesHistory.Add(e);
        // safely raise the event by using ?.Invoke
        ProcessStartedEvent?.Invoke(this, e);
    }

    protected virtual void OnProcessEnded(ProcessEventArgs e) {
        ProcessesHistory.Add(e);
        ProcessEndedEvent?.Invoke(this, e);

        if (ProcessesQueue.TryDequeue(out var nextProcess)) {
            OnProcessStarted(nextProcess);
        }
    }
}

In [ ]:
public class Stove : Appliance {
    class Top {
        public int Index { get; init; }
        public string Something { get; set; }
        public bool IsBusy => !string.IsNullOrEmpty(Something);
    }

    private Top[] Tops { get; } = Enumerable.Range(1, 6)
        .Select(i => new Top {
            Index = i
        })
        .ToArray();
    
    public void Heat(string something, int onTop) {
        var process = new ProcessEventArgs {
            Message = $"Start heating {something} at stove top {onTop}"
        };

        if (Tops[onTop].IsBusy) {
            Console.WriteLine($"Stove top {onTop} is currently heating {Tops[onTop].Something}.");
            ProcessesQueue.Enqueue(process);
            Console.WriteLine("Order enqueued.");
            return;
        }
        
        Tops[onTop].Something = something;
        OnProcessStarted(process);
    }

    public string Remove(int fromTop) {
        if (!Tops[fromTop].IsBusy) {
            Console.WriteLine($"There's nothing on stove top {fromTop}.");
            return null;
        }

        var something = Tops[fromTop].Something;
        Tops[fromTop].Something = null;

        OnProcessEnded(new ProcessEventArgs {
            Message = $"End heating {something} on stove top {fromTop}."
        });

        return something;
    }

    protected override void OnProcessStarted(ProcessEventArgs e) {
        base.OnProcessStarted(e);
    }

    protected override void OnProcessEnded(ProcessEventArgs e) {
        base.OnProcessEnded(e);
    }
}

In [ ]:
public class SmartKitchen {
    public List<Appliance> Appliances { get; } = new();

    public void AddApliance(Appliance appliance) {
        Appliances.Add(appliance);
        appliance.ProcessStartedEvent += LogProcess;
        appliance.ProcessEndedEvent += LogProcess;
    }

    private void LogProcess(object sender, ProcessEventArgs e) {
        Console.WriteLine($"{sender.GetType()}:\n\t{e.Message}\n\t{e.Start}");
    }
}

In [ ]:
var stove = new Stove();
var smartKitchen = new SmartKitchen();
smartKitchen.AddApliance(stove);

stove.Heat("water", 0);

Submission#136+Stove:
	Start heating water at stove top 0
	1/22/2022 4:19:15 PM


In [ ]:
stove.Heat("soup", 0);

Stove top 0 is currently heating water.
Order enqueued.


In [ ]:
stove.Heat("soup", 1);

Submission#136+Stove:
	Start heating soup at stove top 1
	1/22/2022 4:19:15 PM


In [ ]:
stove.Remove(0)

Submission#136+Stove:
	End heating water on stove top 0.
	1/22/2022 4:19:15 PM
Submission#136+Stove:
	Start heating soup at stove top 0
	1/22/2022 4:19:15 PM


water

In [ ]:
public class Oven : Appliance {
    private string Something { get; set; }
    private float Temperature { get; set; }
    public bool IsBusy => !string.IsNullOrEmpty(Something);

    public void Bake(string something, float temperature) {
        var process = new ProcessEventArgs {
            Message = $"Start baking {Something} at {Temperature} degrees."
        };

        if (IsBusy) {
            Console.WriteLine($"Oven is currently baking {Something}.");
            ProcessesQueue.Enqueue(process);
            Console.WriteLine("Order enqueued.");
            return;
        }

        Something = something;
        Temperature = temperature;

        OnProcessStarted(process);
    }

    public string TakeOut() {
        if (!IsBusy) {
            Console.WriteLine("There's nothing in the oven.");
            return null;
        }

        var something = Something;
        Something = null;
        Temperature = 0;

        OnProcessEnded(new ProcessEventArgs {
            Message = $"End baking {something}."
        });

        return something;
    }

    protected override void OnProcessStarted(ProcessEventArgs e) {
        base.OnProcessStarted(e);
    }

    protected override void OnProcessEnded(ProcessEventArgs e) {
        base.OnProcessEnded(e);
    }
}

In [ ]:
var oven = new Oven();
smartKitchen.AddApliance(oven);

oven.Bake("cake", 300);

Submission#141+Oven:
	Start baking  at 0 degrees.
	1/22/2022 4:19:15 PM


In [ ]:
stove.CurrentProcess

Message,Start
Start heating soup at stove top 0,2022-01-22 16:19:15Z


In [ ]:
oven.Bake("turkey", 220);
oven.Bake("pie", 100);

Oven is currently baking cake.
Order enqueued.
Oven is currently baking cake.
Order enqueued.


In [ ]:
oven.TakeOut()

Submission#141+Oven:
	End baking cake.
	1/22/2022 4:19:16 PM
Submission#141+Oven:
	Start baking cake at 300 degrees.
	1/22/2022 4:19:15 PM


cake

In [ ]:
oven.ProcessesHistory

index,Message,Start
0,Start baking at 0 degrees.,2022-01-22 16:19:15Z
1,End baking cake.,2022-01-22 16:19:16Z
2,Start baking cake at 300 degrees.,2022-01-22 16:19:15Z
